In [1]:
import os, sys
src_paths = [os.path.abspath("../"), os.path.abspath("../point-to-pixel-mapping")]
for src_path in src_paths:
    if src_path not in sys.path:
        sys.path.append(src_path)

from datasets.kitti_dataset import KittiDataset

In [2]:
kitti_dataset_path = '/Users/laurenzheidrich/Downloads/fused_dataset'
sequence = '00'
sam_labels_path = '/Users/laurenzheidrich/Downloads/fused_dataset/vfm-labels/sam/00'

dataset = KittiDataset(kitti_dataset_path, 
                       sequence, 
                       sam_labels_path)

labels_folder = '/Users/laurenzheidrich/Downloads/fused_dataset/sequences/00/labels'

In [3]:
from segmentation.reprojection_segmentation import ReprojectionSegmentation
from segmentation.dbscan_segmentation import DbscanSegmentation
from segmentation.spectral_clustering_segmentation import SpectralClusteringSegmentation

visual_segmentation = ReprojectionSegmentation(dataset)
geometrical_segmentation = DbscanSegmentation(dataset)
spectral_segmentation = SpectralClusteringSegmentation(dataset, n_clusters=40, lower_bound=0, upper_bound=25)

### Run baselines

In [4]:
import os
import numpy as np

from tqdm import tqdm

indices = []
visual_labels = []
geometrical_labels = []
spectral_labels = []
gt_labels = []


for i in tqdm(range(0, len(dataset.poses), 1500)):
    indices.append(i)
    visual_labels.append(visual_segmentation.segment_instances(i))
    geometrical_labels.append(geometrical_segmentation.segment_instances(i))
    spectral_labels.append(spectral_segmentation.segment_instances(i))
    labels_file_path = os.path.join(labels_folder, '{}.label'.format(str(i).zfill(6)))
    gt_labels.append(np.fromfile(labels_file_path, dtype=np.uint32))


  0%|          | 0/4 [00:00<?, ?it/s]/Users/laurenzheidrich/miniconda3/envs/o3d/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
 25%|██▌       | 1/4 [01:33<04:40, 93.48s/it]/Users/laurenzheidrich/miniconda3/envs/o3d/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
 50%|█████     | 2/4 [03:12<03:13, 96.94s/it]/Users/laurenzheidrich/miniconda3/envs/o3d/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
 50%|█████     | 2/4 [08:25<08:25, 252.62s/it]


KeyboardInterrupt: 

### Visualize baselines

In [5]:
import random

def generate_random_colors(N):
    colors = []
    for _ in range(N):
        colors.append([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])

    return colors

random_colors = generate_random_colors(500)

In [6]:
import numpy as np
import open3d as o3d

def color_points_by_labels(points, labels):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = []

    for i in range(labels.shape[0]):
        try:
            colors.append(random_colors[int(labels[i]) + 1])
        except:
            colors.append(random_colors[1])
            print(int(labels[i]) + 1)

    pcd.colors = o3d.utility.Vector3dVector(np.vstack(colors) / 255)

    return pcd

In [7]:
index = 1
points = dataset.get_point_cloud(indices[index])

In [ ]:
o3d.visualization.draw_geometries([color_points_by_labels(points, visual_labels[index])])

In [ ]:
o3d.visualization.draw_geometries([color_points_by_labels(points, gt_labels[index])])

In [8]:
o3d.visualization.draw_geometries([color_points_by_labels(points, spectral_labels[index])])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
o3d.visualization.draw_geometries([color_points_by_labels(points, geometrical_labels[index])])

### Evaluation

In [ ]:
import instanseg
from instanseg.metrics import precision, recall, fScore

instanseg.metrics.constants.IOU_THRESHOLD_FULL = 0.5
tp_condition = 'iou'

In [ ]:
import pandas as pd

results_dict = []
for i in tqdm(range(len(spectral_labels))):
    res = {
        'index': i,
        'method': 'visual',
        'precision': precision(visual_labels[i], gt_labels[i], tp_condition),
        'recall': recall(visual_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(visual_labels[i], gt_labels[i], tp_condition)
        
    }
    results_dict.append(res)

    res = {
        'index': i,
        'method': 'geometrical',
        'precision': precision(geometrical_labels[i], gt_labels[i], tp_condition),
        'recall': recall(geometrical_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(geometrical_labels[i], gt_labels[i], tp_condition)
    }
    results_dict.append(res)

    res = {
        'index': i,
        'method': 'spectral',
        'precision': precision(spectral_labels[i], gt_labels[i], tp_condition),
        'recall': recall(spectral_labels[i], gt_labels[i], tp_condition),
        'fscore': fScore(spectral_labels[i], gt_labels[i], tp_condition)
    }
    results_dict.append(res)


In [ ]:
df = pd.DataFrame.from_records(results_dict, index=np.arange(len(results_dict)))

In [ ]:
print(df.tail())

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(df, x='index', y='recall', hue='method')

In [ ]:
sns.lineplot(df, x='index', y='precision', hue='method')

In [ ]:
sns.lineplot(df, x='index', y='fscore', hue='method')